In [5]:
import pyspark
from pyspark.sql import SparkSession, types
from pyspark.conf import Sparkconf
from pyspark.context import SparkContext
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, IntegerType, DateType, StructField, StringType, TimestampType
import pyspark.pandas as ps
import logging, traceback
import requests
import sys
import pandas as pd
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
from dateutil.parser import parse

In [2]:
pyspark.__file__

'/home/camiloms/spark/spark-3.3.3-bin-hadoop3/python/pyspark/__init__.py'

In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

23/09/06 23:35:56 WARN Utils: Your hostname, LAPTOP-C9HBU13M resolves to a loopback address: 127.0.1.1; using 172.21.82.95 instead (on interface eth0)
23/09/06 23:35:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/09/06 23:35:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/09/06 23:36:05 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [ ]:
df_past = spark.read.parquet('gs://import-bucket-camiloms10/raw/players_game_stats_current_year.parquet')

In [ ]:
#df = spark.read \
#    .option("header", "true") \
#    .csv('Sample-Spreadsheet-5000-rows.csv')
#
#df.show()

In [ ]:
#df.write.parquet('zones')

In [ ]:
one_yr_ago = datetime.now() - relativedelta(years=1)

In [ ]:
last_year_end = date(one_yr_ago.date().year, 12, 31)

In [ ]:
last_year_end

In [ ]:
#Getting the players table (Dimension table)

players_table = pd.DataFrame(players.get_players())

In [ ]:
#Getting the teams table (Dimension table)

teams_table = pd.DataFrame(teams.get_teams())

In [ ]:
players_list = players_table['id'].to_list()

In [ ]:
teams_list = teams_table['id'].to_list()

In [ ]:
players_list

In [ ]:
#Getting the regular season stats by team (all of history)

appended_data = []

for team in teams_list:
    team_stats_by_year = teamyearbyyearstats.TeamYearByYearStats(team_id=team,season_type_all_star=SeasonTypeAllStar.regular,per_mode_simple=PerModeSimple.totals)
    current_team_stats = team_stats_by_year.team_stats.get_data_frame()
    appended_data.append(current_team_stats)

team_stats_df = pd.concat(appended_data)

In [ ]:
Seasons = list(team_stats_df['YEAR'].unique())

In [ ]:
Seasons.sort()

In [ ]:
schema = types.StructType([
                         types.StructField("SEASON_ID", types.StringType(), True),
                         types.StructField("PLAYER_ID", types.IntegerType(), True),
                         types.StructField("PLAYER_NAME", types.StringType(), True),
                         types.StructField("NICKNAME", types. StringType(), True),
                         types.StructField("TEAM_ID", types.IntegerType(), True),
                         types.StructField("TEAM_ABBREVIATION", types.StringType(), True),
                         types.StructField("TEAM_NAME", types. StringType(), True),
                         types.StructField("GAME_ID", types.StringType(), True),
                         types.StructField("GAME_DATE", types.StringType(), True),
                         types.StructField("MATCHUP", types.StringType(), True),
                         types.StructField("WL", types.StringType(), True),
                         types.StructField("MIN", types.IntegerType(), True),
                         types.StructField("FGM", types.IntegerType(), True),
                         types.StructField("FGA", types.FloatType(), True),
                         types.StructField("FG_PCT", types.FloatType(), True),
                         types.StructField("FG3M", types.IntegerType(), True),
                         types.StructField("FG3A", types.IntegerType(), True),
                         types.StructField("FG3_PCT", types.FloatType(), True),
                         types.StructField("FTM", types.IntegerType(), True),
                         types.StructField("FTA", types. IntegerType(), True),
                         types.StructField("FT_PCT", types.FloatType(), True),
                         types.StructField("OREB", types.IntegerType(), True),
                         types.StructField("DREB", types.IntegerType(), True),
                         types.StructField("REB", types.IntegerType(), True),
                         types.StructField("AST", types.IntegerType(), True),
                         types.StructField("STL", types.IntegerType(), True),
                         types.StructField("BLK", types.IntegerType(), True),
                         types.StructField("TOV", types.IntegerType(), True),
                         types.StructField("PF", types.IntegerType(), True),
                         types.StructField("PTS", types.IntegerType(), True),
                         types.StructField("PLUS_MINUS", types.IntegerType(), True),
                         types.StructField("VIDEO_AVAILABLE", types.IntegerType(), True)
])

In [ ]:
player_history_data = pd.DataFrame()

for season in Seasons:
    try:
        playerlogdata=playergamelogs.PlayerGameLogs(season_nullable=season)
        playerlogdata=playerlogdata.player_game_logs.get_data_frame()
        playerlogdata=pd.DataFrame(playerlogdata)
        playerlogdata[["FTA","REB","AST","PF"]]= playerlogdata[["FTA","REB","AST","PF"]].fillna(0).astype(int)
        playerlogdata.iteritems = playerlogdata.items
        print(playerlogdata)
        
        #Create PySpark DataFrame from Pandas
        sparkDF=spark.createDataFrame(playerlogdata, schema= schema).show()
        
        #player_history_data[['GAME_DATE']] = player_history_data[['GAME_DATE']].apply(pd.to_datetime)
        print(season)
    except as e:
        print('skipped'+season+'because of'+e)

In [ ]:
player_history_data 

In [ ]:
player_history_data = player_history_data.loc[(player_history_data['GAME_DATE'].dt.date <= last_year_end)]

In [ ]:
player_history_data